# Rapport de cadrage

## Problématique du projet
La consommation électrique varie fortement selon l'heure de la journée, la météo et le calendrier. Ces fluctuations rendent la planification du réseau complexe et augmentent les coûts lorsqu'elle est mal anticipée (surproduction, achat d'énergie à prix élevés).

L'objectif du projet est donc de prédire la consommation électrique horaire jusqu'à 24h à l'avance, à partir des données historiques et de variables explicatives externes (météo, calendrier), afin de permettre une meilleure planification de la production, une réduction des coûts opérationnels et une gestion plus optimale du réseau électrique.

## Définition opérationnelle
Nous formulons le problème comme une tâche de régression de séries temporelles multi-horizon (H = 24h).

- Entrée : séquence historique \(X(t) = [\text{consommation}, \text{météo}, \text{variables calendaires}]\)
- Sortie : séquence future \(Y(t+1:t+24)\)

Objectif : fournir des prévisions robustes pour l'horizon de 24 heures.

## Enjeux techniques
- Capturer les patterns cycliques (journalier, hebdomadaire, saisonnier)
- Intégrer l'influence de la météo
- Gérer le caractère non stationnaire de la série
- Comparer architectures deep learning et baselines statistiques

## Formulation finale
Comment prédire la consommation électrique horaire sur un horizon de 24h, en combinant historiques, météo et informations calendaires, de manière à obtenir des prévisions robustes permettant d'optimiser la gestion du réseau électrique ?

## Contraintes à intégrer
- Saisonniété
- Jour/nuit
- Weekend / semaine
- Jours fériés
- Température et météo
- Granularité horaire